# Morningstar: Entraînement sur Google Colab

Ce notebook permet d'entraîner le modèle Morningstar en utilisant les ressources GPU de Google Colab.

**Étapes:**
1.  Exécuter les cellules d'installation et de configuration.
2.  Exécuter la cellule "Upload Fichier de Données" et téléverser votre fichier de données (ex: `full_dataset.parquet`).
3.  Exécuter les cellules restantes pour préparer les données, entraîner le modèle et sauvegarder/télécharger le résultat.

## 1. Installation des Dépendances et Clonage du Projet

In [ ]:
# Installation des bibliothèques nécessaires
!pip install -q tensorflow pyarrow wandb pandas numpy

# Vérification GPU
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU disponible: {gpu_devices}")
    # Configuration pour éviter les erreurs OOM sur certaines cartes
    try:
        for gpu in gpu_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth activé pour les GPUs.")
    except RuntimeError as e:
        print(f"Erreur lors de l'activation de memory growth: {e}")
else:
    print("Aucun GPU détecté. L'entraînement se fera sur CPU (peut être très lent).")

In [ ]:
# Clonage du dépôt GitHub (contient le code source du modèle et des utilitaires)
!git clone https://github.com/Cabrel10/eva001.git
%cd eva001

# Installation des dépendances spécifiques du projet
!pip install -q .

## 2. Configuration Weights & Biases (W&B)

In [ ]:
# Connexion à W&B (utilise la clé API fournie)
!wandb login a1478933771f0389426436c0de1c39585a5a452c

## 3. Upload Fichier de Données

Exécutez la cellule suivante et utilisez le bouton "Choisir les fichiers" pour téléverser votre fichier de données Parquet (par exemple, `full_dataset.parquet`). Assurez-vous que le nom du fichier correspond à celui utilisé dans la cellule de préparation des données plus bas.

In [ ]:
from google.colab import files

print("Veuillez téléverser votre fichier de données Parquet...")
uploaded = files.upload()

# Vérification - affiche les noms des fichiers téléversés
for fn in uploaded.keys():
  print(f'Fichier "{fn}" téléversé avec succès ({len(uploaded[fn])} bytes)')

# Définir le nom du fichier pour la suite (MODIFIEZ SI NECESSAIRE)
uploaded_filename = list(uploaded.keys())[0] # Prend le premier fichier téléversé
print(f"Nom du fichier retenu pour la préparation: {uploaded_filename}")

## 4. Préparation des Données

Cette section charge le fichier Parquet téléversé, sélectionne les features, gère les valeurs manquantes, normalise les données, crée les séquences et génère les labels (Buy/Sell/Hold) en utilisant la logique du projet.

In [ ]:
import pandas as pd
import numpy as np
import logging
import os

# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Importer les classes nécessaires depuis le projet cloné
try:
    from Morningstar.configs.morningstar_config import MorningstarConfig
    from Morningstar.workflows.morningstar_training import MorningstarTrainingWorkflow
except ImportError as e:
    logger.error(f"Erreur d'importation des modules du projet: {e}")
    logger.error("Assurez-vous que le clonage et l'installation (pip install .) ont réussi.")
    raise

# Chemin vers le fichier uploadé dans Colab
# Assurez-vous que 'uploaded_filename' a été défini correctement dans la cellule précédente
colab_data_path = f"/content/{uploaded_filename}"

if not os.path.exists(colab_data_path):
    logger.error(f"Le fichier de données '{colab_data_path}' n'a pas été trouvé !")
    logger.error("Vérifiez que le fichier a été correctement téléversé et que le nom est correct.")
    raise FileNotFoundError(colab_data_path)

logger.info(f"Utilisation du fichier de données: {colab_data_path}")

# Instancier la configuration
config = MorningstarConfig()

# *** Point crucial : Modifier la configuration pour utiliser le fichier uploadé ***
# Crée une copie ou modifie l'instance pour pointer vers le bon chemin
class ColabConfig(MorningstarConfig):
    DATA_PATH: str = colab_data_path

colab_config = ColabConfig()

# Instancier le workflow avec la configuration adaptée
workflow = MorningstarTrainingWorkflow(colab_config)

# Préparer les données (chargement, features, NaN, scaling, séquences, labels, split)
logger.info("Début de la préparation des données...")
try:
    (train_seq, train_labels), (val_seq, val_labels) = workflow.prepare_data()
    logger.info("Préparation des données terminée.")
    logger.info(f"Shape Train Sequences: {train_seq.shape}, Train Labels: {train_labels.shape}")
    logger.info(f"Shape Validation Sequences: {val_seq.shape}, Validation Labels: {val_labels.shape}")

    # Créer les tuples pour l'entraînement
    train_data = (train_seq, train_labels)
    val_data = (val_seq, val_labels)

except Exception as e:
    logger.error(f"Erreur lors de la préparation des données: {e}")
    import traceback
    traceback.print_exc()
    raise

## 5. Initialisation de l'Expérience W&B

In [ ]:
import wandb

# Initialisation de l'expérience W&B pour le suivi
try:
    wandb.init(project="morningstar-colab-training", entity="cabrelkaka-morningstar", config=colab_config.__dict__)
    logger.info("Wandb initialisé avec succès.")
except Exception as e:
    logger.error(f"Erreur lors de l'initialisation de Wandb: {e}")
    # Continuer sans W&B si l'initialisation échoue ? Ou arrêter ?
    # Pour l'instant, on continue mais on log l'erreur.
    wandb_active = False
else:
    wandb_active = True

## 6. Chargement et Compilation du Modèle

In [ ]:
from Morningstar.model.architecture.morningstar_model import MorningstarTradingModel

# Déterminer input_shape et num_classes à partir des données préparées
input_shape = train_data[0].shape[1:] # (time_window, num_features)
num_classes = train_data[1].shape[1] # Nombre de classes (Buy, Sell, Hold -> 3)

logger.info(f"Initialisation du modèle avec input_shape={input_shape}, num_classes={num_classes}")

# Instancier le modèle
model_instance = MorningstarTradingModel(
    input_shape=input_shape,
    num_classes=num_classes,
    # Passer d'autres paramètres d'architecture si nécessaire depuis colab_config
    cnn_filters=colab_config.cnn_filters,
    lstm_units=colab_config.lstm_units,
    dense_units=colab_config.dense_units
)

# Compiler le modèle
model_instance.compile_model(learning_rate=colab_config.learning_rate)
logger.info("Modèle compilé.")
model_instance.model.summary() # Afficher le résumé du modèle

## 7. Configuration des Callbacks

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Définition des callbacks pour l'entraînement
callbacks = [
    # Sauvegarde le meilleur modèle basé sur la perte de validation
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', verbose=1),
    # Arrête l'entraînement si la perte de validation ne s'améliore pas pendant 15 époques
    EarlyStopping(patience=15, monitor='val_loss', restore_best_weights=True, verbose=1)
]

# Ajouter le callback W&B si l'initialisation a réussi
if wandb_active:
    try:
        from wandb.keras import WandbCallback
        callbacks.append(WandbCallback())
        logger.info("Callback Wandb ajouté.")
    except ImportError:
        logger.warning("Impossible d'importer WandbCallback. Le suivi W&B sera limité.")

logger.info(f"Callbacks configurés: {callbacks}")

## 8. Entraînement du Modèle

L'entraînement peut prendre du temps en fonction de la taille des données et du nombre d'époques.

In [ ]:
logger.info("Début de l'entraînement...")

# Utilisation de MirroredStrategy si plusieurs GPUs sont détectés (facultatif)
strategy = tf.distribute.MirroredStrategy() if len(gpu_devices) > 1 else tf.distribute.get_strategy()
logger.info(f"Utilisation de la stratégie de distribution: {strategy.__class__.__name__}")

with strategy.scope():
    # Ré-instancier et compiler le modèle dans le scope de la stratégie si nécessaire
    # (Keras gère souvent cela automatiquement, mais c'est plus sûr pour certains cas)
    # model_instance_scoped = MorningstarTradingModel(...) 
    # model_instance_scoped.compile_model(...)
    
    history = model_instance.model.fit(
        x=train_data[0], # Séquences d'entraînement
        y=train_data[1], # Labels d'entraînement
        epochs=colab_config.epochs,
        batch_size=colab_config.batch_size,
        validation_data=val_data, # Données de validation (séquences, labels)
        callbacks=callbacks
    )

logger.info("Entraînement terminé.")

# Optionnel: Afficher les métriques finales
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
logger.info(f"Perte finale (entraînement): {final_train_loss:.4f}")
logger.info(f"Perte finale (validation): {final_val_loss:.4f}")

# Terminer l'exécution W&B si active
if wandb_active:
    wandb.finish()

## 9. Sauvegarde sur Google Drive (Optionnel)

In [ ]:
from google.colab import drive
import os

try:
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/Morningstar_Models/' # Chemin sur votre Drive
    os.makedirs(drive_path, exist_ok=True) # Crée le dossier s'il n'existe pas
    
    model_save_path = os.path.join(drive_path, 'best_model_colab.h5')
    
    if os.path.exists('best_model.h5'):
        !cp best_model.h5 "{model_save_path}"
        logger.info(f"Modèle sauvegardé sur Google Drive: {model_save_path}")
    else:
        logger.warning("Le fichier 'best_model.h5' n'a pas été trouvé. Sauvegarde sur Drive annulée.")
        logger.warning("(Cela peut arriver si EarlyStopping n'a pas trouvé de meilleur modèle que l'initial)")

except Exception as e:
    logger.error(f"Erreur lors de la sauvegarde sur Google Drive: {e}")

## 10. Téléchargement du Modèle Entraîné

Exécutez la cellule suivante pour télécharger le fichier `best_model.h5` (contenant les poids du meilleur modèle trouvé pendant l'entraînement) sur votre machine locale.

In [ ]:
from google.colab import files

if os.path.exists('best_model.h5'):
    print("Téléchargement de best_model.h5...")
    files.download('best_model.h5')
else:
    logger.error("Le fichier 'best_model.h5' n'a pas été trouvé pour le téléchargement.")